# 04. Self Oil Station Price Analysis

[1. 셀레니움 설치](#install-selenium) <br>

[Github에서는 toc가 작동안함. nbviewer로 보기👀](https://nbviewer.org/github/solpinetree/ds_study/blob/main/source_code/04.%20Self%20Oil%20Station%20Price%20Analysis.ipynb)

<a id="install-selenium"></a>
## 1. 셀레니움 설치
- 윈도우, mac(intel)
    - ```conda install conda-forge::selenium```
- mac(m-)
    - ```pip install selenium```

In [9]:
from selenium import webdriver
import selenium

In [12]:
driver = webdriver.Chrome()
driver.get("https://www.naver.com")

In [13]:
driver.quit()